In [1]:
# Locations
ROOT = "C:\\OneDrive - Netherlands eScience Center\\Project_Wageningen_iOMEGA"
PATH_MS_DATA = ROOT + "\\Data\\GNPS_all\\"
PATH_SAVE_MODEL = ROOT + "\\Spec2Vec\\models_trained\\"
PATH_SAVE_DATA = ROOT + "\\Spec2Vec\\data\\"
PATH_SPEC2VEC = ROOT + "\\Spec2Vec\\code\\"

In [2]:
import numpy as np
import sys
sys.path.insert(0, PATH_SPEC2VEC)

import helper_functions as functions
import MS_functions

## Import unique-inchi subset

In [3]:
# Import & filter data from unique inchikey dataset
file_json = PATH_MS_DATA + "filtered_data_uniqueInchikey_minpeak10_loss500_2dec_exp001.json"
file_mgf = ROOT + "\\Data\\GNPS_all\\" + "MS_data_allGNPS_uniqueInchikey14_191101.mgf"

spectra, spectra_dict, MS_documents, MS_documents_intensity, spectra_metadata = MS_functions.load_MGF_data(file_mgf = file_mgf,
                                                           file_json = file_json,
                                                           num_decimals = 2,
                                                           min_frag = 0.0, max_frag = 1000.0,
                                                           min_loss = 5.0, max_loss = 500.0,
                                                           min_intensity_perc = 0,
                                                           exp_intensity_filter = 0.01,
                                                           peaks_per_mz = 15/200,
                                                           min_peaks = 10,
                                                           max_peaks = None,
                                                           peak_loss_words = ['peak_', 'loss_'])

Could not find file  C:\OneDrive - Netherlands eScience Center\Project_Wageningen_iOMEGA\Data\GNPS_all\filtered_data_uniqueInchikey_minpeak10_loss500_2dec_exp001.json
Data will be imported from  C:\OneDrive - Netherlands eScience Center\Project_Wageningen_iOMEGA\Data\GNPS_all\MS_data_allGNPS_uniqueInchikey14_191101.mgf


C:\Users\FlorianHuber\Anaconda3\envs\SeSiMe\lib\site-packages\scipy\optimize\minpack.py:787: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
C:\OneDrive - Netherlands eScience Center\Project_Wageningen_iOMEGA\Spec2Vec\code\MS_functions.py:372: RuntimeWarning: overflow encountered in multiply
  return a*np.exp(-b*x)
C:\Users\FlorianHuber\Anaconda3\envs\SeSiMe\lib\site-packages\scipy\optimize\minpack.py:780: RuntimeWarning: overflow encountered in multiply
  pcov = pcov * s_sq


Unclear TypeError for  100  peaks. Use mean intensity as threshold.
[] and y:  []
RuntimeError for  123  peaks. Use mean intensity as threshold.
RuntimeError for  114  peaks. Use mean intensity as threshold.
RuntimeError for  77  peaks. Use mean intensity as threshold.
RuntimeError for  54  peaks. Use mean intensity as threshold.
RuntimeError for  180  peaks. Use mean intensity as threshold.
RuntimeError for  287  peaks. Use mean intensity as threshold.
Unclear TypeError for  301  peaks. Use mean intensity as threshold.
[1.403703] and y:  [33]
Take  10992 spectra out of  12813 .
  Created documents for  1200  of  10992  spectra.No losses detected for:  1221 1430
No losses detected for:  1241 1450
  Created documents for  1500  of  10992  spectra.No losses detected for:  1538 1793
  Created documents for  1600  of  10992  spectra.No losses detected for:  1665 1932
  Created documents for  1700  of  10992  spectra.No losses detected for:  1753 2020
No losses detected for:  1773 2040
  Cr

In [5]:
print("Number of imported spectra:", len(spectra))

Number of imported spectra: 10992


In [45]:
spectra[0].inchi

'"InChI=1S/C26H43NO5/c1-15(4-7-22(30)27-14-23(31)32)18-5-6-19-24-20(9-11-26(18,19)3)25(2)10-8-17(28)12-16(25)13-21(24)29/h15-21,24,28-29H,4-14H2,1-3H3,(H,27,30)(H,31,32)/t15?,16-,17+,18+,19-,20-,21+,24-,25?,26?/m0/s1"'

## Calculate molecular fingerprints

In [7]:
from openbabel import openbabel as ob
from openbabel import pybel
pybel.fps

['ecfp0',
 'ecfp10',
 'ecfp2',
 'ecfp4',
 'ecfp6',
 'ecfp8',
 'fp2',
 'fp3',
 'fp4',
 'maccs']

In [45]:
def mol_fingerprints(spectra_dict, 
                     method = "ecfp6"):
    """ Calculate molecule fingerprints based on given inchi or smiles (using RDkit).
    
    Output: exclude_IDs list with spectra that had no inchi or smiles or problems when deriving fingerprint
    
    Args:
    --------
    spectra_dict: dict
        Dictionary containing all spectrum objects information (peaks, losses, metadata...).
    method: str
        Determine method for deriving molecular fingerprints. Supported choices from openbabel are 'ecfp0',
        'ecfp10', 'ecfp2', 'ecfp4', 'ecfp6', 'ecfp8', 'fp2', 'fp3', 'fp4', 'maccs'. (see "pybel.fps")
    """
    
    # If spectra is given as a dictionary
    keys = []
    exclude_IDs = []
    molecules = []
    
    for key, value in spectra_dict.items():
        mol = 0
        if "inchi" in value["metadata"] and len(value["metadata"]["inchi"]) > 12:
            inchi = value["metadata"]["inchi"]
            if inchi.split('InChI=')[-1][0] == '1':  # only select right format
                mol = 1
                keys.append(key) 
                try:
                    mol = pybel.readstring("inchi", inchi) 
                except:
                    print(key, ' ---- ')
                    print('error handling inchi:', inchi)
                    mol = 0

        if "smiles" in value and mol == 0:  # Smiles but no InChikey or inchi handling failed
            keys.append(key) 
            smiles = value["metadata"]["smiles"]
            if len(smiles) > 5:
                try:
                    mol = pybel.readstring("smi", smiles)
                    if len(mol.atoms)>2:
                        print('Molecule found using smiles:', smiles)
                except:
                    print('error handling smiles:', smiles)
                    mol = 0
    
        if mol == 0: #or mol == 1:
            print("No smiles found for spectra ", key, ".")
            mol = ''
            molecules.append(mol)
            exclude_IDs.append(int(value["id"]))
        
        #"""  
        else:
            #mol = ''
            molecules.append(mol)

        
    fingerprints = []
    for i in range(len(molecules)):
        if molecules[i] is None:
            print("Problem with molecule " + str(spectra_dict[keys[i]]["id"]))
            fp = 0
            exclude_IDs.append(int(spectra_dict[keys[i]]["id"]))
        else:
            try:
                fp = molecules[i].calcfp(method)
                #if len(fp.bits) > 1:
                #    print("Created fingerprint.")
            except:
                fp = 0
                exclude_IDs.append(int(spectra_dict[keys[i]]["id"]))

        fingerprints.append(fp)
    
    return molecules, fingerprints, exclude_IDs

In [ ]:
inchi_control = 'InChI=1S/C26H32O9/c1-11-13-8-17(27)35-22(3,4)14(13)9-16-24(6)15(11)10-23(5)18-19(33-20(23)28)32-12(2)26(30,34-16)25(18,24)21(29)31-7/h8-9,11-12,15-16,18-19,30H,10H2,1-7H3/t11-,12-,15-,16+,18+,19+,23+,24+,25-,26+/m0/s1'
inchi_notwork = 'InChI=1S/C26H32O9/c1-11-13-8-17(27)35-22(3,4)14(13)9-16-24(6)15(11)10-23(5)18-19(33-20(23)28)32-12(2)26(30,34-16)25(18,24)21(29)31-7/h8-9,11-12,15-16,18-19,30H,10H2,1-7H3/t11-,12-,15-,16+,18+,19+,23+,24+,25-,26+/m0/s1'

In [41]:
mol = pybel.readstring("smi", 'cc') 
len(mol.atoms)

2

In [41]:
spectra_dict['1791']

{'id': 1791,
 'filename': [],
 'peaks': [[56.832417, 1112.615723],
  [81.32457, 1466.502075],
  [72.909615, 1483.507202],
  [112.597542, 1542.089233],
  [80.953026, 1575.963013],
  [59.029915, 1601.985596],
  [92.100334, 1698.112793],
  [72.762817, 1716.799927],
  [362.978729, 1749.238525],
  [91.053925, 1769.755493],
  [101.059532, 1776.271851],
  [90.976517, 1851.651001],
  [81.069901, 1894.630005],
  [303.023285, 1897.45166],
  [52.064011, 1950.487183],
  [69.07032, 2042.015503],
  [145.254257, 2128.078857],
  [72.765762, 2131.506348],
  [264.13739, 2348.274658],
  [143.036865, 2352.503174],
  [116.04953, 2394.674805],
  [123.116287, 2413.346191],
  [347.26828, 2416.842041],
  [345.19693, 2525.523926],
  [186.482788, 2695.491699],
  [97.101097, 2744.726074],
  [329.244995, 2843.077148],
  [97.064804, 3111.710205],
  [55.054802, 3179.731934],
  [95.085739, 3394.924072],
  [125.09623, 3518.343506],
  [85.065071, 3534.494629],
  [204.119598, 5719.601562],
  [145.046951, 6424.220703],
 

In [46]:
molecules_ob, fingerprints_ecfp6, exclude_IDs_ob = mol_fingerprints(spectra_dict, 
                                                                    method = "ecfp6")

Molecule found using smiles: C1[C@@H]([C@H](O[C@H]1N2C3=C(C(=O)N=C(N3)N)NC2=O)CO)O
1973  ---- 
error handling inchi: "InChI=1S/C18H15NO5/c20-13-7-5-12(6-8-13)3-1-2-4-17(22)19-16-10-9-14(21)11-15(16)18(23)24/h1-11,20-21H,(H,19,22)(H,23,24)/b3-1,4-2+"
Molecule found using smiles: OC1=CC=C(/C=C/C=C/C(NC2=CC=C(O)C=C2C(O)=O)=O)C=C1
2042  ---- 
error handling inchi: "InChI=1S/C16H17NO3/c18-16(17-9-3-4-10-17)6-2-1-5-13-7-8-14-15(11-13)20-12-19-14/h1-2,5-8,11H,3-4,9-10,12H2/b5-1,6-2+"
Molecule found using smiles: O=C(N1CCCC1)/C=C/C=C/C2=CC=C3OCOC3=C2
2106  ---- 
error handling inchi: "InChI=1S/C11H11Br2N5O/c12-7-4-8(18-9(7)13)10(19)15-3-1-2-6-5-16-11(14)17-6/h1-2,4-5,18H,3H2,(H,15,19)(H3,14,16,17)/b2-1"
Molecule found using smiles: C1=C(NC(=C1Br)Br)C(=O)NCC=CC2=CN=C(N2)N
2765  ---- 
error handling inchi: "InChI=1S/C29H45ClN6O9S/c"
Molecule found using smiles: CC[C@H](C)[C@@H](NC(=O)C(O)Cc1ccc(O)c(Cl)c1)C(=O)N1C2CC(CCC2CC1C(=O)NCCCCNC(=N)N)OS(=O)(=O)O
2826  ---- 
error handling inchi: "InChI=1S

In [47]:
exclude_ob = [np.where(np.array(spectra_metadata)[:,1] == x)[0][0] for x in exclude_IDs_ob]
len(exclude_ob)

2

## note: only 2 fingerprints could not been made
Those spectra will be ignored for the rest of the analysis.